https://github.com/Michedev/VAE_anomaly_detection/blob/master/vae_anomaly_detection/VAE.py

In [1]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score
import sys
sys.path.append('..')
import datasets.load_tabular as NAD
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
import torch.nn.init as weight_init
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader 

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [122]:
arrhy_train_X, arrhy_train_y, arrhy_test_X, arrhy_test_y = NAD.Arrhythmia_train_test_split('/home/sewon/anomaly_detection/datasets/')

In [123]:
scaler = StandardScaler()
scaler.fit(arrhy_train_X)
train = scaler.transform(arrhy_train_X)

In [124]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        # self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_mean = nn.Linear(hidden_dim, latent_dim)
        self.fc_var = nn.Linear(hidden_dim, latent_dim)
        self.actf = nn.ReLU()
        
    def forward(self, x):
        h1 = self.actf(self.linear1(x))
        mean = self.fc_mean(h1)
        log_var = self.fc_var(h1)
        
        return mean, log_var        

In [125]:
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.linear3 = nn.Linear(latent_dim, hidden_dim)
        # self.linear4 = nn.Linear(hidden_dim, hidden_dim)
        self.rc_mean = nn.Linear(hidden_dim, output_dim)
        self.rc_var = nn.Linear(hidden_dim, output_dim)
        self.actf = nn.ReLU()
    
    def forward(self, x):
        h2 = self.actf(self.linear3(x))
        r_mean = self.rc_mean(h2)
        r_log_var = self.rc_var(h2)
        
        return r_mean, r_log_var

In [126]:
class VAEAD(nn.Module):
    def __init__(self, Encoder, Decoder, L = 10):
        super(VAEAD, self).__init__()
        self.encoder = Encoder
        self.decoder = Decoder
        self.L = L # number of monte carlo estimates
        self.prior = Normal(0,1)
        
    def reparameterization(self, mu, std):
        epsilon = torch.rand_like(std).to(device)
        z = mu + std * epsilon
        return z
    
    def forward(self, x):
        pred_result = self.predict(x)
        x = x.unsqueeze(0)
        # log_like = Normal(pred_result['recon_mu'], torch.exp(0.5*pred_result['recon_logvar'])).log_prob(x).mean(
        #     dim=0)  # average over sample dimension
        log_like = -0.5*(((x.repeat((self.L,1,1)) - pred_result['recon_mu'])**2)/torch.exp(pred_result['recon_logvar']) + 
                         (torch.log(torch.tensor(2.0*torch.pi)) + pred_result['recon_logvar']))
        log_like = log_like.mean(dim=0)
        log_like = log_like.mean(dim=0).sum()
        kl_div = -0.5 * torch.sum(1 + pred_result['latent_logvar'] - pred_result['latent_mu'].pow(2) - pred_result['latent_logvar'].exp())
        loss = kl_div - log_like
        return dict(loss = loss, kl_div = kl_div, recon_loss = log_like, **pred_result)
        
    def predict(self,x):
        mu, log_var = self.encoder(x.float())
        z_list = []
        for i in range(self.L):
            z_sample = self.reparameterization(mu, torch.exp(0.5 * log_var))
            z_list.append(z_sample)
        z_mc = torch.cat(z_list, 0)   # (L x batch_size, latent size) 
      
        recon_mu, recon_logvar = self.decoder(z_mc.float())
        recon_mu = recon_mu.view(self.L, *x.shape)
        recon_logvar = recon_logvar.view(self.L, *x.shape)
        
        return dict(latent_mu = mu, latent_logvar = log_var, 
                    recon_mu = recon_mu, recon_logvar = recon_logvar, z = z_mc)
    
    
    def is_anomaly(self, x, alpha = 0.05):
        p = self.reconstruction_prob(x)
        return p < alpha
    
    def reconstruction_prob(self, x):
        with torch.no_grad():
            pred = self.predict(x)
        recon_dist =  Normal(pred['recon_mu'], torch.exp(0.5 * pred['recon_logvar']))
        x = x.unsqueeze(0)
        p = recon_dist.log_prob(x).exp().mean(dim = 0)
        p = p.mean(dim = -1) # vector of shape [batch_size]
        return p
    
            
    def generate(self, batch_size, latent_size):
        z = self.prior.sample((batch_size, latent_size))
        recon_mu, recon_logvar = self.decoder(z.float())
        return recon_mu + torch.exp(0.5 * recon_logvar) * torch.rand_like(recon_logvar)
            

In [127]:
batch_size = 100
epochs = 100

In [128]:
enc = Encoder(arrhy_train_X.shape[1], 400, 200)
dec = Decoder(200, 400, arrhy_train_X.shape[1])

In [129]:
train_loader = DataLoader(dataset = train, batch_size=batch_size, shuffle=False)

In [130]:
model = VAEAD(enc, dec).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

In [131]:
print("Start training VAE...")
model.train()

for epoch in range(epochs):
    overall_loss = 0
    for batch_idx, x in enumerate(train_loader):
        optimizer.zero_grad()
        x = x.to(device)
        pred = model(x)
        loss = pred['loss']
        overall_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    print('Epoch [%d / %d] Average Loss:: %f' % (epoch+1, epochs, overall_loss / (batch_idx*batch_size)))    
    
print("Finish!!")

Start training VAE...
Epoch [1 / 100] Average Loss:: 20.123727
Epoch [2 / 100] Average Loss:: 18.861308
Epoch [3 / 100] Average Loss:: 17.793285
Epoch [4 / 100] Average Loss:: 16.877999
Epoch [5 / 100] Average Loss:: 16.091130
Epoch [6 / 100] Average Loss:: 15.416071
Epoch [7 / 100] Average Loss:: 14.833265
Epoch [8 / 100] Average Loss:: 14.325826
Epoch [9 / 100] Average Loss:: 13.881872
Epoch [10 / 100] Average Loss:: 13.487742
Epoch [11 / 100] Average Loss:: 13.141473
Epoch [12 / 100] Average Loss:: 12.822610
Epoch [13 / 100] Average Loss:: 12.535174
Epoch [14 / 100] Average Loss:: 12.272960
Epoch [15 / 100] Average Loss:: 12.026290
Epoch [16 / 100] Average Loss:: 11.798085
Epoch [17 / 100] Average Loss:: 11.585753
Epoch [18 / 100] Average Loss:: 11.383223
Epoch [19 / 100] Average Loss:: 11.195696
Epoch [20 / 100] Average Loss:: 11.015023
Epoch [21 / 100] Average Loss:: 10.848338
Epoch [22 / 100] Average Loss:: 10.682636
Epoch [23 / 100] Average Loss:: 10.523355
Epoch [24 / 100] Aver

In [132]:
train.shape

(193, 274)

In [133]:
test = scaler.transform(arrhy_test_X)

In [134]:
model.eval()
with torch.no_grad():
    test_X = torch.FloatTensor(test)
    test_X = test_X.to(device)
    recon_prob = model.reconstruction_prob(test_X)

In [135]:
recon_prob

tensor([1.5800, 1.7054, 1.5198, 1.7173, 1.6722, 1.6432, 1.5808, 1.5528, 1.6399,
        1.6529, 1.6908, 1.5701, 1.6562, 1.6062, 1.6653, 1.7243, 1.6342, 1.5196,
        1.6985, 1.6497, 1.6171, 1.6554, 1.6840, 1.6189, 1.6516, 1.6837, 1.6340,
        1.6449, 1.6743, 1.6440, 1.6036, 1.6741, 1.4456, 1.6305, 1.7153, 1.7217,
        1.6942, 1.6928, 1.6821, 1.6763, 1.6076, 1.5559, 1.6663, 1.7303, 1.6264,
        1.3029, 1.5160, 1.7345, 1.6137, 1.6156, 1.6628, 1.6447, 1.7370, 1.6671,
        1.5637, 1.4794, 1.6457, 1.5454, 1.6337, 1.5866, 1.4576, 1.0734, 1.5836,
        1.5472, 1.6791, 1.5982, 1.6099, 1.5516, 1.4423, 1.6267, 1.4677, 1.5752,
        1.6161, 0.9175, 1.3230, 1.6943, 1.5859, 1.6848, 1.3581, 1.6014, 1.4915,
        1.5502, 1.4649, 1.6416, 1.7601, 1.6175, 1.7169, 1.6003, 1.5936, 1.5345,
        1.6269, 1.5826, 1.5347, 1.6338, 1.4529, 1.6182, 1.6184, 1.4809, 1.6862,
        1.6295, 1.7269, 1.6628, 1.6849, 1.6335, 1.7053, 1.4111, 1.7321, 1.5160,
        1.6410, 1.6135, 1.5012, 1.0616, 

In [141]:
f1_score(arrhy_test_y, (recon_prob < 1.6).cpu().numpy().astype(int), average='binary')

0.5257142857142858

In [168]:
from torch.distributions import Normal, kl_divergence
from torch.nn.functional import softplus

In [169]:
def tabular_encoder(input_size: int, latent_size: int):
    """
    Simple encoder for tabular data.
    If you want to feed image to a VAE make another encoder function with Conv2d instead of Linear layers.
    :param input_size: number of input variables
    :param latent_size: number of output variables i.e. the size of the latent space since it's the encoder of a VAE
    :return: The untrained encoder model
    """
    return nn.Sequential(
        nn.Linear(input_size, 500),
        nn.ReLU(),
        nn.Linear(500, 200),
        nn.ReLU(),
        nn.Linear(200, latent_size * 2)  # times 2 because this is the concatenated vector of latent mean and variance
    )


def tabular_decoder(latent_size: int, output_size: int):
    """
    Simple decoder for tabular data.
    :param latent_size: size of input latent space
    :param output_size: number of output parameters. Must have the same value of input_size
    :return: the untrained decoder
    """
    return nn.Sequential(
        nn.Linear(latent_size, 200),
        nn.ReLU(),
        nn.Linear(200, 500),
        nn.ReLU(),
        nn.Linear(500, output_size * 2)
        # times 2 because this is the concatenated vector of reconstructed mean and variance
    )

In [170]:
class VAEAnomalyDetection(nn.Module):
    def __init__(self, input_size, latent_size, L=10):
        super(VAEAnomalyDetection, self).__init__()
        self.L = L
        self.input_size = input_size
        self.latent_size = latent_size
        self.encoder = tabular_encoder(input_size, latent_size)
        self.decoder = tabular_decoder(latent_size, input_size)
        self.prior = Normal(0, 1)
        
    def forward(self, x):
        pred_result = self.predict(x)
        x = x.unsqueeze(0)  # unsqueeze to broadcast input across sample dimension (L)
        log_lik = Normal(pred_result['recon_mu'], pred_result['recon_sigma']).log_prob(x).mean(
            dim=0)  # average over sample dimension
        log_lik = log_lik.mean(dim=0).sum()
        kl = kl_divergence(pred_result['latent_dist'], self.prior).mean(dim=0).sum()
        loss = kl - log_lik
        return dict(loss=loss, kl=kl, recon_loss=log_lik, **pred_result)

    def predict(self, x):
        batch_size = len(x)
        latent_mu, latent_sigma = self.encoder(x).chunk(2, dim=1) #both with size [batch_size, latent_size]
        latent_sigma = softplus(latent_sigma)
        dist = Normal(latent_mu, latent_sigma)
        z = dist.rsample([self.L])  # shape: [L, batch_size, latent_size]
        z = z.view(self.L * batch_size, self.latent_size)
        recon_mu, recon_sigma = self.decoder(z).chunk(2, dim=1)
        recon_sigma = softplus(recon_sigma)
        recon_mu = recon_mu.view(self.L, *x.shape)
        recon_sigma = recon_sigma.view(self.L, *x.shape)
        return dict(latent_dist=dist, latent_mu=latent_mu,
                    latent_sigma=latent_sigma, recon_mu=recon_mu,
                    recon_sigma=recon_sigma, z=z)

    def is_anomaly(self, x, alpha=0.05):
        p = self.reconstructed_probability(x)
        return p < alpha

    def reconstructed_probability(self, x):
        with torch.no_grad():
            pred = self.predict(x)
        recon_dist = Normal(pred['recon_mu'], pred['recon_sigma'])
        x = x.unsqueeze(0)
        p = recon_dist.log_prob(x).exp().mean(dim=0).mean(dim=-1)  # vector of shape [batch_size]
        return p

    def generate(self, batch_size: int=1) -> torch.Tensor:
        z = self.prior.sample((batch_size, self.latent_size))
        recon_mu, recon_sigma = self.decoder(z).chunk(2, dim=1)
        recon_sigma = softplus(recon_sigma)
        return recon_mu + recon_sigma * torch.rand_like(recon_sigma)

In [171]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [172]:
batch_size = 100
epochs = 30
model = VAEAnomalyDetection(arrhy_train_X.shape[1], 200, 10).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

In [173]:
train_loader = DataLoader(dataset = arrhy_train_X, batch_size=batch_size, shuffle=False)

In [174]:
print("Start training VAE...")
model.train()

for epoch in range(epochs):
    overall_loss = 0
    for batch_idx, x in enumerate(train_loader):
    
        optimizer.zero_grad()
        x = x.to(device)
        pred = model(x.float())
        loss = pred['loss']
        overall_loss += loss.item()
        loss.backward()
        optimizer.step()
        
    print("\tEpoch", epoch + 1, "complete!", "\tAverage Loss: ", overall_loss / (batch_idx*batch_size))
    
print("Finish!!")

Start training VAE...
	Epoch 1 complete! 	Average Loss:  3865.231015625
	Epoch 2 complete! 	Average Loss:  694.80337890625
	Epoch 3 complete! 	Average Loss:  344.562509765625
	Epoch 4 complete! 	Average Loss:  245.24439453125
	Epoch 5 complete! 	Average Loss:  204.117783203125
	Epoch 6 complete! 	Average Loss:  165.185107421875
	Epoch 7 complete! 	Average Loss:  131.065634765625
	Epoch 8 complete! 	Average Loss:  105.5690380859375
	Epoch 9 complete! 	Average Loss:  89.9092626953125
	Epoch 10 complete! 	Average Loss:  79.740888671875
	Epoch 11 complete! 	Average Loss:  69.49947509765624
	Epoch 12 complete! 	Average Loss:  60.49935791015625
	Epoch 13 complete! 	Average Loss:  53.7478564453125
	Epoch 14 complete! 	Average Loss:  48.09618896484375
	Epoch 15 complete! 	Average Loss:  43.38744140625
	Epoch 16 complete! 	Average Loss:  39.5595361328125
	Epoch 17 complete! 	Average Loss:  36.28773681640625
	Epoch 18 complete! 	Average Loss:  33.57497192382812
	Epoch 19 complete! 	Average Loss:

In [175]:
model.eval()
with torch.no_grad():
    test_X = torch.FloatTensor(arrhy_test_X)
    test_X = test_X.to(device)
    recon_prob = model.reconstructed_probability(test_X)

In [176]:
recon_prob

tensor([0.6219, 0.6297, 0.6460, 0.6263, 0.6738, 0.6614, 0.6802, 0.6372, 0.6565,
        0.6160, 0.6625, 0.6364, 0.6579, 0.6266, 0.6736, 0.6319, 0.6661, 0.6565,
        0.6027, 0.6768, 0.5837, 0.5936, 0.6638, 0.6184, 0.6330, 0.6468, 0.6544,
        0.6699, 0.6935, 0.6360, 0.6444, 0.6595, 0.6106, 0.6733, 0.6637, 0.6806,
        0.7046, 0.6497, 0.6976, 0.6531, 0.6691, 0.6464, 0.6333, 0.6719, 0.6449,
        0.6936, 0.6342, 0.7040, 0.6937, 0.6119, 0.6744, 0.6910, 0.6764, 0.6251,
        0.6422, 0.5764, 0.6467, 0.6084, 0.7056, 0.5983, 0.6251, 0.4510, 0.5668,
        0.6740, 0.6404, 0.7098, 0.6810, 0.6393, 0.5552, 0.6511, 0.5285, 0.6080,
        0.6384, 0.6241, 0.5699, 0.6551, 0.6522, 0.6724, 0.6278, 0.5994, 0.6702,
        0.6490, 0.6086, 0.6343, 0.6968, 0.6276, 0.6715, 0.6706, 0.6730, 0.6370,
        0.6488, 0.6321, 0.6095, 0.7021, 0.6425, 0.6264, 0.6512, 0.5972, 0.6394,
        0.6330, 0.6772, 0.6714, 0.6517, 0.7051, 0.6141, 0.6613, 0.6868, 0.6161,
        0.6595, 0.6118, 0.6247, 0.5420, 

In [190]:
f1_score(arrhy_test_y, (recon_prob < 0.63).cpu().numpy().astype(int), average='binary')

0.5952380952380952